In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler


In [2]:
# ROOT = r"D:\Randome shit\TourX\ml-services\Geolife Trajectories 1.3\Data"

# def load_all_geolife(root_path):
#     all_data = []

#     for user in os.listdir(root_path):
#         user_path = os.path.join(root_path, user, "Trajectory")

#         if os.path.exists(user_path):

#             for file in os.listdir(user_path):
#                 if file.endswith(".plt"):
#                     file_path = os.path.join(user_path, file)

#                     df = pd.read_csv(
#                         file_path,
#                         skiprows=6,
#                         header=None,
#                         names=["lat","lon","unused","alt","date","date2","time"]
#                     )

#                     df["user_id"] = user
#                     df["trajectory_id"] = file

#                     all_data.append(df)

#     return pd.concat(all_data, ignore_index=True)

# df = load_all_geolife(ROOT)

# print("Loaded shape:", df.shape)
# df.head()


In [2]:
df = pd.read_csv("D:\\Randome shit\\TourX\\ml-services\\geolife_processed.csv")

In [5]:
df.head()

,lat,lon,unused,alt,date,date2,time,user_id,trajectory_id,timestamp,time_gap,prev_lat,prev_lon,distance_delta,speed,prev_speed,acceleration,bearing,prev_bearing,direction_change
0,39.984686,116.318417,0,492.0,39744.120313,2008-10-23,02:53:15,0,20081023025304.plt,2008-10-23 02:53:15,5.0,39.984683,116.318450,2.831299,0.566260,0.586148,-0.003978,276.766339,126.922277,149.844061
1,39.984688,116.318385,0,492.0,39744.120370,2008-10-23,02:53:20,0,20081023025304.plt,2008-10-23 02:53:20,5.0,39.984686,116.318417,2.735434,0.547087,0.566260,-0.003835,274.663284,276.766339,2.103055
2,39.984655,116.318263,0,492.0,39744.120428,2008-10-23,02:53:25,0,20081023025304.plt,2008-10-23 02:53:25,5.0,39.984688,116.318385,11.023008,2.204602,0.547087,0.331503,250.555849,274.663284,24.107435
3,39.984611,116.318026,0,493.0,39744.120486,2008-10-23,02:53:30,0,20081023025304.plt,2008-10-23 02:53:30,5.0,39.984655,116.318263,20.776544,4.155309,2.204602,0.390141,256.379828,250.555849,5.823979
4,39.984608,116.317761,0,493.0,39744.120544,2008-10-23,02:53:35,0,20081023025304.plt,2008-10-23 02:53:35,5.0,39.984611,116.318026,22.580319,4.516064,4.155309,0.072151,269.153609,256.379828,12.773781


In [3]:
# Convert Excel serial to numeric
df["date"] = pd.to_numeric(df["date"], errors="coerce")

# Convert to timestamp
df["timestamp"] = pd.to_datetime(
    df["date"],
    unit="D",
    origin="1899-12-30"
)

# Clean precision noise
df["timestamp"] = df["timestamp"].dt.round("1s")

# Drop bad rows
df = df.dropna(subset=["timestamp"])

# Sort correctly
df = df.sort_values(["user_id", "trajectory_id", "timestamp"])

print("Timestamp fixed")
df.head()


Timestamp fixed


,lat,lon,unused,alt,date,date2,time,user_id,trajectory_id,timestamp
0,39.984702,116.318417,0,492.0,39744.120185,2008-10-23,02:53:04,0,20081023025304.plt,2008-10-23 02:53:04
1,39.984683,116.318450,0,492.0,39744.120255,2008-10-23,02:53:10,0,20081023025304.plt,2008-10-23 02:53:10
2,39.984686,116.318417,0,492.0,39744.120313,2008-10-23,02:53:15,0,20081023025304.plt,2008-10-23 02:53:15
3,39.984688,116.318385,0,492.0,39744.120370,2008-10-23,02:53:20,0,20081023025304.plt,2008-10-23 02:53:20
4,39.984655,116.318263,0,492.0,39744.120428,2008-10-23,02:53:25,0,20081023025304.plt,2008-10-23 02:53:25


In [4]:
df["time_gap"] = (
    df.groupby(["user_id","trajectory_id"])["timestamp"]
      .diff()
      .dt.total_seconds()/3600
)


In [5]:
df["prev_lat"] = df.groupby(["user_id","trajectory_id"])["lat"].shift()
df["prev_lon"] = df.groupby(["user_id","trajectory_id"])["lon"].shift()

In [6]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # meters
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi/2)**2 + \
        np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2

    return 2 * R * np.arcsin(np.sqrt(a))

df["distance_delta"] = haversine(
    df["prev_lat"], df["prev_lon"],
    df["lat"], df["lon"]
)

In [8]:
df["speed"] = df["distance_delta"] / df["time_gap"]


In [9]:
df["prev_speed"] = df.groupby(["user_id","trajectory_id"])["speed"].shift()

df["acceleration"] = (
    (df["speed"] - df["prev_speed"]) / df["time_gap"]
)


In [10]:
def compute_bearing(lat1, lon1, lat2, lon2):
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    delta_lon = np.radians(lon2 - lon1)

    x = np.sin(delta_lon) * np.cos(lat2)
    y = (
        np.cos(lat1) * np.sin(lat2)
        - np.sin(lat1) * np.cos(lat2) * np.cos(delta_lon)
    )

    bearing = np.degrees(np.arctan2(x, y))
    bearing = (bearing + 360) % 360
    return bearing

df["bearing"] = compute_bearing(
    df["prev_lat"],
    df["prev_lon"],
    df["lat"],
    df["lon"]
)


In [11]:
df["prev_bearing"] = df.groupby(
    ["user_id","trajectory_id"]
)["bearing"].shift()

df["direction_change"] = abs(df["bearing"] - df["prev_bearing"])

df["direction_change"] = np.minimum(
    df["direction_change"],
    360 - df["direction_change"]
)


In [12]:
df = df.dropna()

print("After cleaning:", df.shape)


After cleaning: (24213481, 20)


In [13]:
df.head()

,lat,lon,unused,alt,date,date2,time,user_id,trajectory_id,timestamp,time_gap,prev_lat,prev_lon,distance_delta,speed,prev_speed,acceleration,bearing,prev_bearing,direction_change
2,39.984686,116.318417,0,492.0,39744.120313,2008-10-23,02:53:15,0,20081023025304.plt,2008-10-23 02:53:15,0.001389,39.984683,116.318450,2.831299,2038.535312,2110.131857,-5.154951e+04,276.766339,126.922277,149.844061
3,39.984688,116.318385,0,492.0,39744.120370,2008-10-23,02:53:20,0,20081023025304.plt,2008-10-23 02:53:20,0.001389,39.984686,116.318417,2.735434,1969.512810,2038.535312,-4.969620e+04,274.663284,276.766339,2.103055
4,39.984655,116.318263,0,492.0,39744.120428,2008-10-23,02:53:25,0,20081023025304.plt,2008-10-23 02:53:25,0.001389,39.984688,116.318385,11.023008,7936.565593,1969.512810,4.296278e+06,250.555849,274.663284,24.107435
5,39.984611,116.318026,0,493.0,39744.120486,2008-10-23,02:53:30,0,20081023025304.plt,2008-10-23 02:53:30,0.001389,39.984655,116.318263,20.776544,14959.111972,7936.565593,5.056233e+06,256.379828,250.555849,5.823979
6,39.984608,116.317761,0,493.0,39744.120544,2008-10-23,02:53:35,0,20081023025304.plt,2008-10-23 02:53:35,0.001389,39.984611,116.318026,22.580319,16257.829620,14959.111972,9.350767e+05,269.153609,256.379828,12.773781


In [ ]:
import os

BASE_PATH = r"D:\Randome shit\TourX\ml-services"

csv_path = os.path.join(BASE_PATH, "geolife_processed.csv")

print("Saving CSV...")
df.to_csv(csv_path, index=False)


print("All files saved successfully!")


Saving CSV...
All files saved successfully!


In [26]:
# import os
# import pyarrow
# BASE_PATH = r"D:\Randome shit\TourX\ml-services"
# parquet_path = os.path.join(BASE_PATH, "geolife_processed.parquet")

# # df.to_parqueta
#     parquet_path,
#     engine="pyarrow",
#     compression="snappy",
#     index=False
# )

# print("Saved in fastest & smallest format (Parquet + Snappy)")


In [12]:
features = [
    "distance_delta",
    "speed",
    "acceleration",
    "time_gap",
    "direction_change"
]

df_model = df[["user_id", "trajectory_id"] + features].copy()

df_model.head()


,user_id,trajectory_id,distance_delta,speed,acceleration,time_gap,direction_change
0,0,20081023025304.plt,2.831299,0.566260,-0.003978,5.0,149.844061
1,0,20081023025304.plt,2.735434,0.547087,-0.003835,5.0,2.103055
2,0,20081023025304.plt,11.023008,2.204602,0.331503,5.0,24.107435
3,0,20081023025304.plt,20.776544,4.155309,0.390141,5.0,5.823979
4,0,20081023025304.plt,22.580319,4.516064,0.072151,5.0,12.773781


In [13]:
df = df[df["time_gap"] > 0]


In [14]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

,lat,lon,unused,alt,date,date2,time,user_id,trajectory_id,timestamp,time_gap,prev_lat,prev_lon,distance_delta,speed,prev_speed,acceleration,bearing,prev_bearing,direction_change
0,39.984686,116.318417,0,492.000000,39744.120313,2008-10-23,02:53:15,0,20081023025304.plt,2008-10-23 02:53:15,5.0,39.984683,116.318450,2.831299,0.566260,0.586148,-0.003978,276.766339,126.922277,149.844061
1,39.984688,116.318385,0,492.000000,39744.120370,2008-10-23,02:53:20,0,20081023025304.plt,2008-10-23 02:53:20,5.0,39.984686,116.318417,2.735434,0.547087,0.566260,-0.003835,274.663284,276.766339,2.103055
2,39.984655,116.318263,0,492.000000,39744.120428,2008-10-23,02:53:25,0,20081023025304.plt,2008-10-23 02:53:25,5.0,39.984688,116.318385,11.023008,2.204602,0.547087,0.331503,250.555849,274.663284,24.107435
3,39.984611,116.318026,0,493.000000,39744.120486,2008-10-23,02:53:30,0,20081023025304.plt,2008-10-23 02:53:30,5.0,39.984655,116.318263,20.776544,4.155309,2.204602,0.390141,256.379828,250.555849,5.823979
4,39.984608,116.317761,0,493.000000,39744.120544,2008-10-23,02:53:35,0,20081023025304.plt,2008-10-23 02:53:35,5.0,39.984611,116.318026,22.580319,4.516064,4.155309,0.072151,269.153609,256.379828,12.773781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24213476,40.914867,111.710500,0,3802.493438,39521.152731,2008-03-14,03:39:56,181,20080314025755.plt,2008-03-14 03:39:56,112.0,40.915433,111.711450,101.698458,0.908022,0.849449,0.000523,231.714749,234.423756,2.709007
24213477,40.914267,111.710333,0,3795.931759,39521.153669,2008-03-14,03:41:17,181,20080314025755.plt,2008-03-14 03:41:17,81.0,40.914867,111.710500,68.171002,0.841617,0.908022,-0.000820,191.855062,231.714749,39.859687
24213478,40.912467,111.710667,0,3795.931759,39521.154884,2008-03-14,03:43:02,181,20080314025755.plt,2008-03-14 03:43:02,105.0,40.914267,111.710333,202.101290,1.924774,0.841617,0.010316,172.033395,191.855062,19.821667
24213479,40.911517,111.711317,0,3779.527559,39521.155185,2008-03-14,03:43:28,181,20080314025755.plt,2008-03-14 03:43:28,26.0,40.912467,111.710667,118.921029,4.573886,1.924774,0.101889,152.657698,172.033395,19.375697


In [16]:
df = df.dropna()


In [17]:
print("Any inf left:", np.isinf(df[features]).sum().sum())
print("Any NaN left:", df[features].isna().sum().sum())


Any inf left: 0
Any NaN left: 0


In [18]:
scaler = MinMaxScaler()

df_model = df[["user_id", "trajectory_id"] + features].copy()
df_model[features] = scaler.fit_transform(df_model[features])

print("Scaling complete")


Scaling complete


In [19]:
def create_sequences_limited(df, features, seq_len, max_per_traj=2000):
    sequences = []

    grouped = df.groupby(["user_id", "trajectory_id"])

    for _, group in grouped:
        data = group[features].values

        if len(data) < seq_len:
            continue

        count = 0
        for i in range(len(data) - seq_len):
            sequences.append(data[i:i+seq_len])
            count += 1

            if count >= max_per_traj:
                break

    return np.array(sequences)


In [20]:
SEQ_LEN = 20

X_sequences = create_sequences_limited(
    df_model,
    features,
    SEQ_LEN,
    max_per_traj=1500
)

print("Shape:", X_sequences.shape)


Shape: (12332162, 20, 5)


In [21]:
import numpy as np

# Convert to float32 (CRITICAL for memory)
X_sequences = X_sequences.astype(np.float32)

print("Total sequences:", len(X_sequences))
print("Memory approx (GB):", X_sequences.nbytes / 1024**3)

# Sample manageable amount first
sample_size = 800000  # Start safe (800k)

np.random.seed(42)
indices = np.random.choice(len(X_sequences), sample_size, replace=False)

X_sample = X_sequences[indices]

print("Sample shape:", X_sample.shape)


Total sequences: 12332162
Memory approx (GB): 4.5940883457660675
Sample shape: (800000, 20, 5)


In [22]:
split = int(0.9 * len(X_sample))

X_train = X_sample[:split]
X_val = X_sample[split:]

print("Train shape:", X_train.shape)
print("Val shape:", X_val.shape)


Train shape: (720000, 20, 5)
Val shape: (80000, 20, 5)


In [24]:
np.save("X_val.npy", X_val)

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.callbacks import EarlyStopping

SEQ_LEN = 20
FEATURE_DIM = 5

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQ_LEN, FEATURE_DIM)),
    LSTM(32),
    RepeatVector(SEQ_LEN),
    LSTM(32, return_sequences=True),
    LSTM(64, return_sequences=True),
    TimeDistributed(Dense(FEATURE_DIM))
])

model.compile(optimizer="adam", loss="mse")

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

model.summary()


d:\Randome shit\TourX\ml-services\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 20, 64)         │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 20, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 20, 32)         │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 20, 64)         │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 20, 5)          │           325 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 63,813 (249.27 KB)

 Trainable params: 63,813 (249.27 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
history = model.fit(
    X_train,
    X_train,
    epochs=12,
    batch_size=512,   # larger batch = safer
    validation_data=(X_val, X_val),
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/12
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 196s 135ms/step - loss: 0.0051 - val_loss: 0.0041
Epoch 2/12
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 186s 132ms/step - loss: 0.0035 - val_loss: 0.0030
Epoch 3/12
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 180s 128ms/step - loss: 0.0026 - val_loss: 0.0023
Epoch 4/12
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 168s 119ms/step - loss: 0.0020 - val_loss: 0.0018
Epoch 5/12
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 173s 123ms/step - loss: 0.0016 - val_loss: 0.0015
Epoch 6/12
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 184s 131ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 7/12
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 178s 127ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 8/12
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 172s 122ms/step - loss: 9.2393e-04 - val_loss: 8.6846e-04
Epoch 9/12
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 186s 132ms/step - loss: 7.7893e-04 - val_loss: 7.0552e-04
Epoch 10/12
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 184s 131ms/step - loss: 6.5274e-04 - val_loss: 5.8071e-04
Epoch 11/12
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 169s 120ms/s

In [26]:
reconstructed = model.predict(X_val, batch_size=512)

errors = np.mean((X_val - reconstructed)**2, axis=(1,2))

threshold = np.mean(errors) + 3 * np.std(errors)

print("Anomaly Threshold:", threshold)


157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step
Anomaly Threshold: 0.002870709


In [27]:
np.save("threshold.npy", threshold)

In [29]:
model.save("geolife_lstm_autoencoder.keras")

import joblib
joblib.dump(scaler, "geolife_scaler.save")

print("Model and scaler saved.")


Model and scaler saved.


In [30]:
reconstructed = model.predict(X_val, batch_size=512)

errors = np.mean((X_val - reconstructed)**2, axis=(1,2))

anomalies = errors > threshold

print("Total sequences:", len(errors))
print("Anomalies detected:", anomalies.sum())
print("Anomaly percentage:", anomalies.mean())


157/157 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step
Total sequences: 80000
Anomalies detected: 1609
Anomaly percentage: 0.0201125


In [ ]:
X_test = X_val.copy()

# Inject spike in speed
X_test[0, :, 1] = 1.0  # max speed spike

reconstructed_test = model.predict(X_test[:10])
errors_test = np.mean((X_test[:10] - reconstructed_test)**2, axis=(1,2))

print("Errors after injection:", errors_test.astype(float))
print("Threshold:", threshold)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Errors after injection: [2.0251040e-01 6.7491014e-06 2.8697934e-03 2.3168004e-04 2.5682340e-03
 4.0550146e-04 1.4221946e-04 4.8200469e-04 1.0408448e-04 2.9616393e-05]
Threshold: 0.002870709


In [ ]:
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model("geolife_lstm_autoencoder.keras")


X_val = np.load("X_val.npy")         # validation sequences
threshold = float(np.load("threshold.npy"))  # saved threshold


X_test = X_val.copy()


X_test[0, :, 1] = 1.0   # strong anomaly in first sequence

reconstructed_test = model.predict(X_test[:10])


errors_test = np.mean((X_test[:10] - reconstructed_test)**2, axis=(1,2))


print("\nErrors after injection:")
for i, e in enumerate(errors_test):
    print(f"Sequence {i}: {e:.10f}")

print("\nThreshold:", threshold)


anomalies = errors_test > threshold

print("\nAnomaly Detection Results:")
for i, flag in enumerate(anomalies):
    print(f"Sequence {i}: {'ANOMALY' if flag else 'Normal'}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step

Errors after injection:
Sequence 0: 0.2025104016
Sequence 1: 0.0000067491
Sequence 2: 0.0028697934
Sequence 3: 0.0002316800
Sequence 4: 0.0025682340
Sequence 5: 0.0004055015
Sequence 6: 0.0001422195
Sequence 7: 0.0004820047
Sequence 8: 0.0001040845
Sequence 9: 0.0000296164

Threshold: 0.0028707089368253946

Anomaly Detection Results:
Sequence 0: ANOMALY
Sequence 1: Normal
Sequence 2: Normal
Sequence 3: Normal
Sequence 4: Normal
Sequence 5: Normal
Sequence 6: Normal
Sequence 7: Normal
Sequence 8: Normal
Sequence 9: Normal
